In [2]:
import math
from decimal import Decimal, getcontext
getcontext().prec = 30

In [3]:
class Vector(object):
    
    CANNOT_NORMALIZE_ZERO_VECTOR_MSG = "Cannot normalize the zero vector"
    
    def __init__(self, coordinates):
        try:
            if not coordinates:
                raise ValueError
            self.coordinates = tuple(Decimal(x) for x in coordinates)
            self.dimension = len(coordinates)

        except ValueError:
            raise ValueError('The coordinates must be nonempty')

        except TypeError:
            raise TypeError('The coordinates must be an iterable')


    def __str__(self):
        return 'Vector: {}'.format(self.coordinates)


    def __eq__(self, v):
        return self.coordinates == v.coordinates
    
    def plus(self,v):
        new_coordinates = [x+y for x, y in zip(self.coordinates,v.coordinates)]
        return Vector(new_coordinates)
    
    def minus(self,v):
        return Vector([x-y for x,y in zip(self.coordinates, v.coordinates)])
    
    def times_scalar(self, c):
        return Vector([Decimal(c)*x for x in self.coordinates])
    
    def magnitude(self):
        return Decimal(math.sqrt(sum([x**2 for x in self.coordinates])))
    
    def normalize(self):
        try:
            return self.times_scalar(Decimal('1.0') / self.magnitude())
        except ZeroDivisionError:
            raise Exeception(self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG)
        
            
    def dot_product(self,w):
        return float("{:.3f}".format(sum([x*y for x, y in zip(self.coordinates,w.coordinates)])))
    
    def angle_with(self,w,inDegrees=False):   
        try:
            u1 = self.normalize()
            u2 = w.normalize()
            theta_in_radians = math.acos(u1.dot_product(u2))
            if inDegrees:
                degrees_per_radian = 189. / math.pi
                return theta_in_radians * degrees_per_radian
            else:
                return theta_in_radians
        except Exception as e:
            raise e
            
    def check_orthogonal(self,w,tolerance=1e-10):
        return abs(self.dot_product(w)) < tolerance
    
    def is_zero(self, tolerance=1e-10):
        return self.magnitude() < tolerance
    
    def check_parallel(self,w):
        return (self.is_zero() or 
                w.is_zero() or 
                self.angle_with(w) == 0 or 
                self.angle_with(w) == math.pi)
    
    def x(self,w):
        try:
            assert self.dimension == 3 and w.dimension == 3
            x1 = self.coordinates[0]
            y1 = self.coordinates[1]
            z1 = self.coordinates[2]
            
            x2 = w.coordinates[0]
            y2 = w.coordinates[1]
            z2 = w.coordinates[2]
            
            a = y1*z2 - y2*z1
            b = -(x1*z2 - x2*z1)
            c = x1*y2 - x2*y1
            return Vector([a,b,c])        
        except ZeroDivisionError:
            raise Exeception(self.CANNOT_NORMALIZE_ZERO_VECTOR_MSG)
            
    def get_parallelogram_area(self,w):
        cross_product_vector = self.x(w)
        return math.sqrt(sum([x**2 for x in cross_product_vector.coordinates]))
    
    def get_triangle_area(self,w):
        cross_product_vector = self.x(w)
        return 0.5*math.sqrt(sum([x**2 for x in cross_product_vector.coordinates]))

In [4]:
class Line(object):

    NO_NONZERO_ELTS_FOUND_MSG = 'No nonzero elements found'

    def __init__(self, normal_vector=None, constant_term=None):
        self.dimension = 2

        if not normal_vector:
            all_zeros = ['0']*self.dimension
            normal_vector = Vector(all_zeros)
        self.normal_vector = normal_vector

        if not constant_term:
            constant_term = Decimal('0')
        self.constant_term = Decimal(constant_term)

        self.set_basepoint()


    def set_basepoint(self):
        try:
            n = self.normal_vector
            c = self.constant_term
            basepoint_coords = ['0']*self.dimension

            initial_index = Line.first_nonzero_index(n)
            initial_coefficient = n[initial_index]

            basepoint_coords[initial_index] = c/initial_coefficient
            self.basepoint = Vector(basepoint_coords)

        except Exception as e:
            if str(e) == Line.NO_NONZERO_ELTS_FOUND_MSG:
                self.basepoint = None
            else:
                raise e


    def __str__(self):

        num_decimal_places = 3

        def write_coefficient(coefficient, is_initial_term=False):
            coefficient = round(coefficient, num_decimal_places)
            if coefficient % 1 == 0:
                coefficient = int(coefficient)

            output = ''

            if coefficient < 0:
                output += '-'
            if coefficient > 0 and not is_initial_term:
                output += '+'

            if not is_initial_term:
                output += ' '

            if abs(coefficient) != 1:
                output += '{}'.format(abs(coefficient))

            return output

        n = self.normal_vector

        try:
            initial_index = Line.first_nonzero_index(n)
            terms = [write_coefficient(n[i], is_initial_term=(i==initial_index)) + 'x_{}'.format(i+1)
                     for i in range(self.dimension) if round(n[i], num_decimal_places) != 0]
            output = ' '.join(terms)

        except Exception as e:
            if str(e) == self.NO_NONZERO_ELTS_FOUND_MSG:
                output = '0'
            else:
                raise e

        constant = round(self.constant_term, num_decimal_places)
        if constant % 1 == 0:
            constant = int(constant)
        output += ' = {}'.format(constant)

        return output
    
    def check_parallel(self, ell):
        n1 = self.normal_vector
        n2 = ell.normal_vector
        return n1.check_parallel(n2)

    
    def __eq__(self, ell):
        
        if self.normal_vector.is_zero():
            if not ell.normal_vector.is_zero():
                return False
            else:
                diff = self.constant_term - ell.constant_term
                return MyDecimal(diff).is_near_zero()
        elif ell.normal_vector.is_zero():
            return False
        
        if not self.check_parallel(ell):
            return False
        
        x0 = self.basepoint
        y0 = ell.basepoint
        basepoint_difference = x0.minus(y0)
        
        n = self.normal_vector
        return basepoint_difference.check_orthogonal(n)
    
    
    def compute_intersection(self, l):
        pass


    @staticmethod
    def first_nonzero_index(iterable):
        for k, item in enumerate(iterable):
            if not MyDecimal(item).is_near_zero():
                return k
        raise Exception(Line.NO_NONZERO_ELTS_FOUND_MSG)


class MyDecimal(Decimal):
    def is_near_zero(self, eps=1e-10):
        return abs(self) < eps

In [10]:
# Question #1
#4.046  * x + 2.836 * y = 1.21
#10.115 * x + 7.09  * y = 3.025
A = 4.046
B = 2.836
k = 1.21

v1 = Vector([4.046,2.836])
n1 = v1.normalize()
str(n1)


# Find Intersection

# No intersection

# Same Line

TypeError: unsupported operand type(s) for /: 'decimal.Decimal' and 'float'